In [ ]:
#Created By: Rowan Converse
#Date: 2023-03-07
#Purpose: Consensus analysis of USFWS biologist labels of imagery from Bosque del Apache and Maxwell NWR

In [1]:
#Imports
import pandas as pd

In [2]:
#Analysis annotations
path = "/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/Dissertation/1_Chapter/consensus/data/20230307_expertanalysislabels.csv"
with open(path) as f:
  df = pd.read_csv(f)

#Refined annotations
refinedpath = "/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/CV4Ecology/Prototyping/Data/Labels/train_val_test/20230307_expertconsensuslabels.csv"
with open(refinedpath) as f:
  ref = pd.read_csv(f)

In [6]:
#Refined annotation statistics
print(len(ref))
print(ref["category"].value_counts())

2375
Mallard                                                               1688
Northern Pintail                                                       262
Canadian Goose                                                         140
Other                                                                   70
Sandhill Crane                                                          52
['Mallard' 'Other']                                                     33
American Wigeon                                                         22
['Mallard' 'Northern Pintail']                                          20
['Northern Pintail' 'Other']                                            15
['American Wigeon' 'Mallard']                                           15
['American Wigeon' 'Mallard' 'Northern Pintail' 'Other' 'Teal']          9
['American Wigeon' 'Other']                                              7
Gadwall                                                                  5
['Gadwall' 'Mallard'

In [ ]:
#INDIVIDUAL LABELER METRICS
#Goal: report avg count, ID consensus across images, break down by morphological group; also min/max individual vs group consensus
#Positional error: create confusion matrix, report commission error, omission error, RMSE, MAE -- bounding boxes, comparing each labeler to the consensus

#Score agreement with consensus class label

#Score %IOU agreement with consensus box 

In [ ]:
#OUTPUTS

#Line graph of average identification consensus across all images (y; % agreement) by individual (x). One for expert, one for crowdsourced.  
#Bar graph of consensus (y; % agreement) by morphology (x; duck/goose/crane). Two bars per class, one for expert, one for crowdsourced
#Line graph of average count consensus across all images (y; % agreement) by individual (x). One for expert, one for crowdsourced.  
#Line graph of consensus (y; % agreement) by density (x; consensus # of individuals per image). One line for expert, one line for crowdsourced.
